# Capstone Project

### Version 7

Final version.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported!')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [2]:
pip install lxml

     |████████████████████████████████| 5.5MB 2.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.



### Scraping of table from specified URL

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url, header=0,flavor=['lxml'])
# tables[0]

### Conversion of table to dataframe

In [4]:
df = pd.DataFrame(tables[0])

In [5]:
df.shape

(180, 3)

In [6]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Removal of rows with no assigned borough

In [7]:
df1 = df[df.Borough != 'Not assigned']

In [8]:
df1.shape

(103, 3)

In [9]:
df1.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df1.reset_index(drop=True).head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Checking for duplicate postal codes

In [11]:
len(df1['Postal Code'].unique())

103

##### The number of Postal Codes is equal to the number of rows of df1, this implies that there are no duplicates.

### Checking for missing neighborhoods

In [12]:
df1.count()

Postal Code     103
Borough         103
Neighborhood    103
dtype: int64

##### The number of values is equal to the number of rows in df1, this implies that here are no missing neighborhoods.

### Size of the resulting dataframe of neighborhoods

In [13]:
df1.shape

(103, 3)

##### The dataframe contains 103 rows.

### Reading the csv file with the Postal Code coordinates

In [14]:
df3 = pd.read_csv('Geospatial_Coordinates.csv')
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df3.shape

(103, 3)

In [16]:
df4 = pd.merge(df1, df3, how='left', on='Postal Code')

### Resulting dataframe with latitude and longitude

In [17]:
df4.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Exploration and clustering of neighborhoods in Toronto

In [18]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Here is a map of the area under study

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

### Creation of new dataframe of Boroughs that contain Toronto

A new data frame with the Boroughs that contain the word Toronto was created in order to work 
with the neighborhoods because Folium does not provide venue information for some of the 
neighborhoods in the complete Toronto area. Therefore, from now on, the work shown is done
with the reduced number of Boroughs.

In [20]:
df5 = df4[df4['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [21]:
df5 = df5.sort_values(by='Neighborhood')

In [22]:
df5.reset_index(drop=True).head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
1,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
2,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
3,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [23]:
df5.shape

(39, 5)

The work will be done with 39 neighborhoods.

In [24]:
latlon = df5[['Latitude','Longitude']].values

In [25]:
latlon[0:5,:]

array([[ 43.6447708, -79.3733064],
       [ 43.6368472, -79.4281914],
       [ 43.6627439, -79.321558 ],
       [ 43.6289467, -79.3944199],
       [ 43.6579524, -79.3873826]])

In [26]:
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [27]:
neigh = len(latlon)
distances = np.empty([39,39])
for i in range(0,neigh):
    for j in range(0,neigh):
        if i == j:
            distances[i,j] = np.inf
        else:
            lat1 = latlon[i,0]
            lon1 = latlon[i,1]
            lat2 = latlon[j,0]
            lon2 = latlon[j,1]
            distances[i,j] = distance(lat1, lon1, lat2, lon2)

In [28]:
dist_min = np.empty([39])
for i in range(0,neigh):
    dist_min[i] = min(distances[i,:])

In [29]:
dist_min

array([0.22270717, 1.40684384, 0.8459053 , 1.66766087, 0.51255249,
       1.40300209, 0.64508797, 0.18151811, 0.94381041, 0.94381041,
       1.58527051, 0.15033369, 1.48042065, 0.68508037, 0.70746365,
       1.56172505, 0.8459053 , 1.05522873, 1.70164277, 1.40684384,
       1.20179543, 0.99339106, 1.56172505, 0.51255249, 1.22839001,
       0.30099349, 1.20179543, 0.99339106, 1.9375388 , 0.50945114,
       1.2673442 , 0.22270717, 1.59834516, 1.40279269, 1.20204816,
       1.98897242, 1.7923045 , 0.15033369, 0.85022024])

In [30]:
min(dist_min)

0.1503336883953562

The minimum distance between neighborhoods is 150 m

In [31]:
dist_min < 0.5

array([ True, False, False, False, False, False, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False,  True, False])

There are 5 neoghborhoods with a minimum distance of less than 500 m to other neighborhoods.

In [32]:
dist_ave = np.empty([39])
for i in range(0,neigh):
    distances[i,i] = 0
    dist_ave[i] = sum(distances[i,:])/(neigh-1)

In [33]:
from statistics import mean
mean(dist_ave)

4.944361252336709

The average distance between neighborhoods is 4.94 Km. I will stick to a radius of 500 m.

### Here is a map of the 39 neighborhoods

In [34]:
# Superimposing neighborhoods in part of Toronto
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Beginning of analysis with Foursquare

In [35]:
CLIENT_ID = '3W1A2N3QRJCNIBMRQEHS5AX0P4JGOKFTZCCBOO0X5CRXCEFF' # your Foursquare ID
CLIENT_SECRET = 'AHW0EUIOGFD1TNT0BGOB3Q1ERYXRZHTC2XKGSIAHTGMZZRFZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3W1A2N3QRJCNIBMRQEHS5AX0P4JGOKFTZCCBOO0X5CRXCEFF
CLIENT_SECRET:AHW0EUIOGFD1TNT0BGOB3Q1ERYXRZHTC2XKGSIAHTGMZZRFZ


### Exploring 39 neighborhoods in Toronto

In [36]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
LIMIT = 100
radius = 500
toronto_venues = getNearbyVenues(names=df5['Neighborhood'],
                                   latitudes=df5['Latitude'],
                                   longitudes=df5['Longitude']
                                  )

Berczy Park
Brockton, Parkdale Village, Exhibition Place
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Central Bay Street
Christie
Church and Wellesley
Commerce Court, Victoria Hotel
Davisville
Davisville North
Dufferin, Dovercourt Village
First Canadian Place, Underground city
Forest Hill North & West, Forest Hill Road Park
Garden District, Ryerson
Harbourfront East, Union Station, Toronto Islands
High Park, The Junction South
India Bazaar, The Beaches West
Kensington Market, Chinatown, Grange Park
Lawrence Park
Little Portugal, Trinity
Moore Park, Summerhill East
North Toronto West, Lawrence Park
Parkdale, Roncesvalles
Queen's Park, Ontario Provincial Government
Regent Park, Harbourfront
Richmond, Adelaide, King
Rosedale
Roselawn
Runnymede, Swansea
St. James Town
St. James Town, Cabbagetown
Stn A PO Boxes
Studio District
Summerhill West, Ra

In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(1623, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
1,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
2,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
3,Berczy Park,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
4,Berczy Park,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [39]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,17,17,17,17,17,17
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


### Checking the total number of neighborhoods under study

In [40]:
len(toronto_venues['Neighborhood'].unique())

39

In [41]:
toronto_venues.shape

(1623, 7)

### Analysis of each neighborhood

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# rename column Neighborhood
toronto_onehot.rename(columns={"Neighborhood":"Neighbor"},inplace=True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighbor,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [43]:
toronto_onehot.shape

(1623, 237)

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped[0:5]

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighbor,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,2,0,0,0,1,1,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,2,0,0,0,0,2,0,0,0,0,1,3,5,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
1,"Brockton, Parkdale Village, Exhibition Place",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,3,0

In [45]:
toronto_grouped.set_index('Neighborhood', inplace=True)

### Number of venues per neighborhood

In [46]:
toronto_grouped.sum(axis=1)

Neighborhood
Berczy Park                                                                                                    57
Brockton, Parkdale Village, Exhibition Place                                                                   23
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                           15
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     17
Central Bay Street                                                                                             65
Christie                                                                                                       17
Church and Wellesley                                                                                           81
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                 

In [47]:
toronto_grouped.shape

(39, 236)

### Procedure for gathering the 237 neighborhoods in 10 main neighborhoods

trans_mat = pd.read_csv('trans_matrix.csv',sep=';')
trans_mat.head()

trans_mat.shape

toronto_grouped.columns

trans_mat.set_index(toronto_grouped.columns, inplace=True)

trans_mat.head()

toronto_regrouped = toronto_grouped.dot(trans_mat)

toronto_regrouped.shape

### Loading of the resulting dataframe toronto_regrouped dataframe

In [60]:
toronto_regrouped = pd.read_csv('toronto_regrouped.csv')

In [61]:
toronto_regrouped.head()

,Neighborhood,Arts & enterteinment,College & university,Event,Food,Nightlife spot,Outdoors & recreation,Professional & other places,Residence,Shop & service,Travel & transport
0,Berczy Park,3,0,0,25,8,4,1,0,14,1
1,"Brockton, Parkdale Village, Exhibition Place",1,0,0,10,2,1,3,0,6,0
2,"Business reply mail Processing Centre, South C...",0,0,0,4,1,4,3,0,3,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,0,0,2,1,1,0,0,1,13
4,Central Bay Street,1,0,0,49,3,2,4,0,5,1


In [62]:
toronto_regrouped.sum()

Neighborhood                   Berczy ParkBrockton, Parkdale Village, Exhibit...
Arts & enterteinment                                                          67
College & university                                                           4
Event                                                                          0
Food                                                                         864
Nightlife spot                                                               150
Outdoors & recreation                                                        101
Professional & other places                                                  101
Residence                                                                      0
Shop & service                                                               268
Travel & transport                                                            63
dtype: object

### Addition of location information to the toronto_grouped dataframe

In [63]:
df5.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
14,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
32,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [64]:
df6 = df5[['Neighborhood']]

In [65]:
df6['Latitude'] = 50 * (df5['Latitude'] - df5['Latitude'].min()) / (df5['Latitude'].max() - df5['Latitude'].min())
df6['Longitude'] = 50 * (df5['Longitude'] - df5['Longitude'].min()) / (df5['Longitude'].max() - df5['Longitude'].min())

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
df6.head()

,Neighborhood,Latitude,Longitude
5,Berczy Park,7.986016,29.031516
14,"Brockton, Parkdale Village, Exhibition Place",3.987179,14.695142
38,"Business reply mail Processing Centre, South C...",17.056578,42.548586
32,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,23.516511
6,Central Bay Street,14.638431,25.354707


In [67]:
toronto_grouped = toronto_regrouped
toronto_grouped.head()

,Neighborhood,Arts & enterteinment,College & university,Event,Food,Nightlife spot,Outdoors & recreation,Professional & other places,Residence,Shop & service,Travel & transport
0,Berczy Park,3,0,0,25,8,4,1,0,14,1
1,"Brockton, Parkdale Village, Exhibition Place",1,0,0,10,2,1,3,0,6,0
2,"Business reply mail Processing Centre, South C...",0,0,0,4,1,4,3,0,3,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,0,0,2,1,1,0,0,1,13
4,Central Bay Street,1,0,0,49,3,2,4,0,5,1


In [68]:
toronto_grouped = pd.merge(df6[['Neighborhood','Latitude','Longitude']], toronto_grouped, how='left', on='Neighborhood')
toronto_grouped.head()

,Neighborhood,Latitude,Longitude,Arts & enterteinment,College & university,Event,Food,Nightlife spot,Outdoors & recreation,Professional & other places,Residence,Shop & service,Travel & transport
0,Berczy Park,7.986016,29.031516,3,0,0,25,8,4,1,0,14,1
1,"Brockton, Parkdale Village, Exhibition Place",3.987179,14.695142,1,0,0,10,2,1,3,0,6,0
2,"Business reply mail Processing Centre, South C...",17.056578,42.548586,0,0,0,4,1,4,3,0,3,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,23.516511,0,0,0,2,1,1,0,0,1,13
4,Central Bay Street,14.638431,25.354707,1,0,0,49,3,2,4,0,5,1


In [69]:
toronto_grouped.shape

(39, 13)

### Cluster neighborhoods

In [70]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 4, 3, 4, 1, 4, 1, 1, 2, 2, 4, 1, 2, 1, 1, 4, 3, 0, 2, 0, 2, 2,
       4, 0, 0, 1, 2, 2, 4, 1, 0, 1, 0, 2, 4, 3, 0, 1, 0], dtype=int32)

This is a much better result of clustering than the one that was made with a radius of 500 m. In
this case, Kmeans was able to separate the neighborhoods in similar clusters.

In [71]:
# add clustering labels
toronto_grouped.drop('Latitude',1,inplace=True)
toronto_grouped.drop('Longitude',1,inplace=True)
toronto_grouped = pd.merge(df5[['Neighborhood','Latitude','Longitude']], toronto_grouped, how='left', on='Neighborhood')
toronto_merged = toronto_grouped
toronto_merged.insert(1, 'Cluster Labels', kmeans.labels_)

toronto_merged.head() # check the last columns!

,Neighborhood,Cluster Labels,Latitude,Longitude,Arts & enterteinment,College & university,Event,Food,Nightlife spot,Outdoors & recreation,Professional & other places,Residence,Shop & service,Travel & transport
0,Berczy Park,0,43.644771,-79.373306,3,0,0,25,8,4,1,0,14,1
1,"Brockton, Parkdale Village, Exhibition Place",4,43.636847,-79.428191,1,0,0,10,2,1,3,0,6,0
2,"Business reply mail Processing Centre, South C...",3,43.662744,-79.321558,0,0,0,4,1,4,3,0,3,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",4,43.628947,-79.394420,0,0,0,2,1,1,0,0,1,13
4,Central Bay Street,1,43.657952,-79.387383,1,0,0,49,3,2,4,0,5,1


In [72]:
df_labels = toronto_merged.groupby('Cluster Labels').count()
df_labels[['Neighborhood']]

,Neighborhood
Cluster Labels,
0,9
1,10
2,9
3,3
4,8


### Here is a map of the different clusters

In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Top ten venues for each neighborhood

In [74]:
toronto_grouped.drop('Latitude',1,inplace=True)
toronto_grouped.drop('Longitude',1,inplace=True)
toronto_grouped.drop('Cluster Labels',1,inplace=True)

In [75]:
toronto_grouped.head()

,Neighborhood,Arts & enterteinment,College & university,Event,Food,Nightlife spot,Outdoors & recreation,Professional & other places,Residence,Shop & service,Travel & transport
0,Berczy Park,3,0,0,25,8,4,1,0,14,1
1,"Brockton, Parkdale Village, Exhibition Place",1,0,0,10,2,1,3,0,6,0
2,"Business reply mail Processing Centre, South C...",0,0,0,4,1,4,3,0,3,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,0,0,2,1,1,0,0,1,13
4,Central Bay Street,1,0,0,49,3,2,4,0,5,1


In [76]:
#toronto_grouped.insert(1, 'Latitude',df6['Latitude'])
#toronto_grouped.insert(1, 'Longitude',df6['Longitude'])
#toronto_merged.drop('Cluster Labels',1,inplace=True)

In [77]:
toronto_merged.head()

,Neighborhood,Arts & enterteinment,College & university,Event,Food,Nightlife spot,Outdoors & recreation,Professional & other places,Residence,Shop & service,Travel & transport
0,Berczy Park,3,0,0,25,8,4,1,0,14,1
1,"Brockton, Parkdale Village, Exhibition Place",1,0,0,10,2,1,3,0,6,0
2,"Business reply mail Processing Centre, South C...",0,0,0,4,1,4,3,0,3,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,0,0,2,1,1,0,0,1,13
4,Central Bay Street,1,0,0,49,3,2,4,0,5,1


In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Food,Shop & service,Nightlife spot,Outdoors & recreation,Arts & enterteinment,Travel & transport,Professional & other places,Residence,Event,College & university
1,"Brockton, Parkdale Village, Exhibition Place",Food,Shop & service,Professional & other places,Nightlife spot,Outdoors & recreation,Arts & enterteinment,Travel & transport,Residence,Event,College & university
2,"Business reply mail Processing Centre, South C...",Outdoors & recreation,Food,Shop & service,Professional & other places,Travel & transport,Nightlife spot,Residence,Event,College & university,Arts & enterteinment
3,"CN Tower, King and Spadina, Railway Lands, Har...",Travel & transport,Food,Shop & service,Outdoors & recreation,Nightlife spot,Residence,Professional & other places,Event,College & university,Arts & enterteinment
4,Central Bay Street,Food,Shop & service,Professional & other places,Nightlife spot,Outdoors & recreation,Travel & transport,Arts & enterteinment,Residence,Event,College & university


### Cluster examination

In [80]:
toronto_merged = neighborhoods_venues_sorted
toronto_merged.insert(1, 'Cluster Labels', kmeans.labels_)
toronto_merged.head()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,0,Food,Shop & service,Nightlife spot,Outdoors & recreation,Arts & enterteinment,Travel & transport,Professional & other places,Residence,Event,College & university
1,"Brockton, Parkdale Village, Exhibition Place",4,Food,Shop & service,Professional & other places,Nightlife spot,Outdoors & recreation,Arts & enterteinment,Travel & transport,Residence,Event,College & university
2,"Business reply mail Processing Centre, South C...",3,Outdoors & recreation,Food,Shop & service,Professional & other places,Travel & transport,Nightlife spot,Residence,Event,College & university,Arts & enterteinment
3,"CN Tower, King and Spadina, Railway Lands, Har...",4,Travel & transport,Food,Shop & service,Outdoors & recreation,Nightlife spot,Residence,Professional & other places,Event,College & university,Arts & enterteinment
4,Central Bay Street,1,Food,Shop & service,Professional & other places,Nightlife spot,Outdoors & recreation,Travel & transport,Arts & enterteinment,Residence,Event,College & university


Cluster 0. Central zone: shopping-recreation.

In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(2, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Food,Shop & service,Nightlife spot,Outdoors & recreation,Arts & enterteinment,Travel & transport,Professional & other places,Residence,Event,College & university
17,"Kensington Market, Chinatown, Grange Park",Food,Shop & service,Outdoors & recreation,Nightlife spot,Professional & other places,Travel & transport,Arts & enterteinment,Residence,Event,College & university
19,"Little Portugal, Trinity",Food,Nightlife spot,Shop & service,Outdoors & recreation,Arts & enterteinment,Professional & other places,Travel & transport,Residence,Event,College & university
23,"Queen's Park, Ontario Provincial Government",Food,Professional & other places,Arts & enterteinment,Shop & service,Nightlife spot,Outdoors & recreation,College & university,Travel & transport,Residence,Event
24,"Regent Park, Harbourfront",Food,Shop & service,Professional & other places,Outdoors & recreation,Nightlife spot,Arts & enterteinment,Travel & transport,Residence,Event,College & university
30,"St. James Town, Cabbagetown",Food,Shop & service,Nightlife spot,Outdoors & recreation,Professional & other places,Arts & enterteinment,Travel & transport,Residence,Event,College & university
32,Studio District,Food,Shop & service,Nightlife spot,Professional & other places,Outdoors & recreation,Travel & transport,Residence,Event,College & university,Arts & enterteinment
36,"The Danforth West, Riverdale",Food,Shop & service,Nightlife spot,Professional & other places,Outdoors & recreation,Travel & transport,Residence,Event,College & university,Arts & enterteinment
38,"University of Toronto, Harbord",Food,Nightlife spot,Shop & service,Professional & other places,College & university,Arts & enterteinment,Travel & transport,Residence,Outdoors & recreation,Event


Cluster 1. Downtown: business-recreation.

In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(2, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Food,Shop & service,Professional & other places,Nightlife spot,Outdoors & recreation,Travel & transport,Arts & enterteinment,Residence,Event,College & university
6,Church and Wellesley,Food,Shop & service,Nightlife spot,Professional & other places,Outdoors & recreation,Travel & transport,Arts & enterteinment,Residence,Event,College & university
7,"Commerce Court, Victoria Hotel",Food,Nightlife spot,Shop & service,Professional & other places,Travel & transport,Outdoors & recreation,Arts & enterteinment,Residence,Event,College & university
11,"First Canadian Place, Underground city",Food,Nightlife spot,Travel & transport,Professional & other places,Arts & enterteinment,Shop & service,Outdoors & recreation,Residence,Event,College & university
13,"Garden District, Ryerson",Food,Shop & service,Professional & other places,Outdoors & recreation,Nightlife spot,Arts & enterteinment,Travel & transport,College & university,Residence,Event
14,"Harbourfront East, Union Station, Toronto Islands",Food,Outdoors & recreation,Nightlife spot,Shop & service,Arts & enterteinment,Travel & transport,Professional & other places,Residence,Event,College & university
25,"Richmond, Adelaide, King",Food,Shop & service,Professional & other places,Nightlife spot,Arts & enterteinment,Travel & transport,Outdoors & recreation,Residence,Event,College & university
29,St. James Town,Food,Shop & service,Nightlife spot,Professional & other places,Outdoors & recreation,Travel & transport,Arts & enterteinment,Residence,Event,College & university
31,Stn A PO Boxes,Food,Shop & service,Nightlife spot,Professional & other places,Outdoors & recreation,Arts & enterteinment,Travel & transport,Residence,Event,College & university
37,"Toronto Dominion Centre, Design Exchange",Food,Travel & transport,Shop & service,Nightlife spot,Arts & enterteinment,Professional & other places,Outdoors & recreation,Residence,Event,College & university


Cluster 2. Northen zone: Residential-recreational area.

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(2, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Davisville,Food,Shop & service,Professional & other places,Outdoors & recreation,Nightlife spot,Travel & transport,Residence,Event,College & university,Arts & enterteinment
9,Davisville North,Food,Shop & service,Professional & other places,Travel & transport,Outdoors & recreation,Residence,Nightlife spot,Event,College & university,Arts & enterteinment
12,"Forest Hill North & West, Forest Hill Road Park",Food,Shop & service,Outdoors & recreation,Travel & transport,Residence,Professional & other places,Nightlife spot,Event,College & university,Arts & enterteinment
18,Lawrence Park,Travel & transport,Professional & other places,Outdoors & recreation,Shop & service,Residence,Nightlife spot,Food,Event,College & university,Arts & enterteinment
20,"Moore Park, Summerhill East",Professional & other places,Outdoors & recreation,Travel & transport,Shop & service,Residence,Nightlife spot,Food,Event,College & university,Arts & enterteinment
21,"North Toronto West, Lawrence Park",Food,Shop & service,Professional & other places,Travel & transport,Outdoors & recreation,Residence,Nightlife spot,Event,College & university,Arts & enterteinment
26,Rosedale,Outdoors & recreation,Travel & transport,Shop & service,Residence,Professional & other places,Nightlife spot,Food,Event,College & university,Arts & enterteinment
27,Roselawn,Shop & service,Outdoors & recreation,Food,Travel & transport,Residence,Professional & other places,Nightlife spot,Event,College & university,Arts & enterteinment
33,"Summerhill West, Rathnelly, South Hill, Forest...",Food,Nightlife spot,Travel & transport,Shop & service,Professional & other places,Residence,Outdoors & recreation,Event,College & university,Arts & enterteinment


Cluster 3. Eastern zone: Recreational area.

In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(2, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Business reply mail Processing Centre, South C...",Outdoors & recreation,Food,Shop & service,Professional & other places,Travel & transport,Nightlife spot,Residence,Event,College & university,Arts & enterteinment
16,"India Bazaar, The Beaches West",Food,Shop & service,Professional & other places,Nightlife spot,Outdoors & recreation,Arts & enterteinment,Travel & transport,Residence,Event,College & university
35,The Beaches,Outdoors & recreation,Shop & service,Nightlife spot,Travel & transport,Residence,Professional & other places,Food,Event,College & university,Arts & enterteinment


Cluster 4. Western zone: Residential area.

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(2, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Brockton, Parkdale Village, Exhibition Place",Food,Shop & service,Professional & other places,Nightlife spot,Outdoors & recreation,Arts & enterteinment,Travel & transport,Residence,Event,College & university
3,"CN Tower, King and Spadina, Railway Lands, Har...",Travel & transport,Food,Shop & service,Outdoors & recreation,Nightlife spot,Residence,Professional & other places,Event,College & university,Arts & enterteinment
5,Christie,Food,Shop & service,Outdoors & recreation,Nightlife spot,Travel & transport,Residence,Professional & other places,Event,College & university,Arts & enterteinment
10,"Dufferin, Dovercourt Village",Shop & service,Nightlife spot,Food,Professional & other places,Outdoors & recreation,Arts & enterteinment,Travel & transport,Residence,Event,College & university
15,"High Park, The Junction South",Food,Shop & service,Outdoors & recreation,Nightlife spot,Arts & enterteinment,Travel & transport,Residence,Professional & other places,Event,College & university
22,"Parkdale, Roncesvalles",Food,Shop & service,Outdoors & recreation,Nightlife spot,Arts & enterteinment,Travel & transport,Residence,Professional & other places,Event,College & university
28,"Runnymede, Swansea",Food,Shop & service,Professional & other places,Nightlife spot,Arts & enterteinment,Travel & transport,Residence,Outdoors & recreation,Event,College & university
34,"The Annex, North Midtown, Yorkville",Food,Shop & service,Outdoors & recreation,Nightlife spot,Arts & enterteinment,Travel & transport,Residence,Professional & other places,Event,College & university
